# 10 - Modules and Packages

## Making a Hierarchical Package of Modules
Making a package structure is simple. Just organize your code as you wish on the filesystem and make sure that every directory defines an __init__.py file.

## Controlling the Import of Everything
You want precise control when the user goes import *.

Define a variable __all__ in your module that explicitly lists the exported names.

In [1]:
import os

code = '''
def spam():
    pass

def grok():
    pass

blah = 42

# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']

'''

file_name = "somemodule.py"
with open(file_name, "w") as target_file:
    target_file.write(code)


In [3]:
%%bash

cat somemodule.py


def spam():
    pass

def grok():
    pass

blah = 42

# Only export 'spam' and 'grok'
__all__ = ['spam', 'grok']



In [10]:
from somemodule import *

"spam" in globals() and "grok" in globals()

True

In [11]:
"blah" in globals()

False

In [12]:
import somemodule as sm

"blah" in vars(sm)

True

So by using __all__ we prevent blah from entering globals.

## Importing Package Submodules Using Relative Names

In [ ]:
# mypackage/A/spam.py

from mypackage.A import grok # OK
from . import grok           # OK
import grok                  # Error (not found)


The downside of using an absolute name, such as mypackage.A, is that it hardcodes the top-level package name into your source code. If you ever changed the name of the package, you would have to go through all of your files and fix the source code.

The . and .. syntax on the import statement might look funny, but think of it as specifying a directory name. . means look in the current directory and ..B means look in the ../B directory.

## Splitting a Module into Multiple Files
A program module can be split into separate files by turning it into a package.

## Making Separate Directories of Code Import Under a Common Namespace

Instead of having each part installed as a separated named package, you would like all of the parts to join together under a common package prefix.

You would like to define a top-level Python package that serves as a namespace for a large collection of separately maintained subpackages.


In [1]:
setup = '''
foo-package/
    spam/
        blah.py
 
bar-package/
    spam/
        grok.py
'''

In [2]:
import os

def create_file(file_path, contents):
    dire_name = os.path.dirname(file_path)
    if not os.path.exists(dire_name):
        os.makedirs(dire_name)

    with open(file_path, "w") as target_file:
        target_file.write(contents)

        
spam_file = os.path.join("foo-package", "spam", "blah.py")
spam_contents = "a = 1"
create_file(spam_file, spam_contents)

grok_file = os.path.join("bar-package", "spam", "grok.py")
grok_contents = "a = 2"
create_file(grok_file, grok_contents)


Observe that there is no __init__.py file in either directory. Now watch what happens if you add both foo-package and bar-package to the Python module path and try some imports:

In [3]:
import sys
sys.path.extend(['foo-package', 'bar-package'])
import spam.blah
import spam.grok

In [6]:
spam.blah.a, spam.grok.a

(1, 2)

The mechanism at work here is a feature known as a 'namespace package'. Essentially, a namespace package is a special kind of package designed for merging different directories of code together under a common namespace.

The key to making a namespace package is to make sure there are no __init__.py files in the top-level directory that is to serve as the common namespace. The missing __init__.py file causes an interesting thing to happen on package import. Instead of
causing an error, the interpreter instead starts creating a list of all directories that happen to contain a matching package name. A special namespace package module is then created and a read-only copy of the list of directories is stored in its __path__ variable.

In [8]:
import spam
spam.__path__

_NamespacePath(['foo-package\\spam', 'bar-package\\spam'])

## Reloading Modules
To reload a previously loaded module, use imp.reload()

In [12]:
import spam
import imp

imp.reload(spam)

<module 'spam' (namespace)>

In [13]:
import sys

set(sys.modules) & set(globals())

{'imp', 'os', 'spam', 'sys'}

##  Making a Directory or Zip File Runnable As a Main Script
If your application program has grown into multiple files, you can put it into its own directory and add a __main__.py file. If __main__.py is present, you can simply run the Python interpreter on the top-level directory.

## Reading Datafiles Within a Package

The pkgutil.get_data() function is meant to be a high-level tool for getting a datafile regardless of where or how a package has been installed. It will simply 'work' and return the file contents back to you as a byte string. It can get messy working with the __file__ variable.


In [ ]:
import pkgutil
data = pkgutil.get_data(__package__, 'somedata.dat')

## Adding Directories to sys.path
You can add them through the use of the PYTHONPATH environment variable. The second approach is to create a .pth file. This .pth file needs to be placed into one of Python’s site-packages directories.

## Importing Modules Using a Name Given in a String

In [15]:
import importlib

math = importlib.import_module('math')
math.sin(2)

0.9092974268256817

In [16]:
mod = importlib.import_module('urllib.request')
u = mod.urlopen('http://www.python.org')

## Patching Modules on Import
You want to patch or apply decorators to functions in an existing module. However, you only want to do it if the module actually gets imported and used elsewhere.